# Récupération jeu de données

L'objectif de ce notebook

Pour la documentation api Spotify



*   https://spotifypy.readthedocs.io/en/latest/api.html
*   https://github.com/spotipy-dev/spotipy/blob/master/examples/search.py
*   https://pypi.org/project/spotipy/








## Installation de l'api SPOTIFY

In [1]:
 # Installation des packages pour Spotify
#pip install spotipy 

## Import des librairies

In [2]:
from function.utils import tqdm, glob, json

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

## Récupération des tokens

/!\ Attention /!\  
Pour pouvoir récuperer les access tokens de spotify il est impératif d'avoir un compte free ou premium.  
De plus avoir créer son APP.  
Par la suite, vous obtiendrez les client_id et client_id_secret permettant l'accès à l'API et le bon fonctionnement du code.

In [3]:
from function.function import get_token

PATH_TOKEN = "../data/token/*"
PATH_JSON = "../data/json_file/"
PATH_TXT = "../data/txt/"

liste_txt_file = glob.glob(PATH_TOKEN)

GENIUS_ACCESS_TOKEN = None
SPOTIFY_client_id = None
SPOTIFY_client_secret = None

GENIUS_ACCESS_TOKEN, SPOTIFY_client_id, SPOTIFY_client_secret = get_token(liste_txt_file)


In [4]:
from function.dictionnary import get_dictionnary

dictionnaire_song = get_dictionnary(PATH_JSON, "dictionnaire_song.json")
liste_artiste = get_dictionnary(PATH_JSON, "dictionnaire_artiste.json")
data_artiste_titre = get_dictionnary(PATH_JSON, "data_artiste_titre.json")

In [5]:
#identifiant spotify
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=SPOTIFY_client_id,
                                                           client_secret=SPOTIFY_client_secret))

data_artiste_titre = list(data_artiste_titre)

if len(data_artiste_titre) == 0:
    for nom_artiste in tqdm.tqdm(liste_artiste):
            results = sp.search(q=nom_artiste, limit=50) # requete de la recherche
            if 'tracks' in results and 'items' in results['tracks']:
                Info_titre = [(idx, nom_artiste, data['name']) for idx, data in enumerate(results['tracks']['items']) ]
            data_artiste_titre.extend(Info_titre)
    with open(PATH_JSON+"data_artiste_titre.txt", "w") as json_file:
        json.dump(data_artiste_titre, json_file, indent=4)
    

In [ ]:
from function.dictionnary import update_dictionnary

headers = {"User-Agent": "Mozilla/5.0"}

for idx, artist_name, song_title in tqdm.tqdm(data_artiste_titre):

    # Si l'artiste est dans le dictionnaire et que le titre est dans le dictionnaire de l'artist
    if artist_name in dictionnaire_song and song_title in dictionnaire_song[artist_name]:

        # Si les paroles sont None tu mets à jour
        if dictionnaire_song[artist_name][song_title]["lyrics"] == None:
            dictionnaire_song = update_dictionnary(song_title, artist_name, GENIUS_ACCESS_TOKEN, dictionnaire_song, headers)
        # Sinon tu passes
        continue
    else:
        dictionnaire_song = update_dictionnary(song_title, artist_name, GENIUS_ACCESS_TOKEN, dictionnaire_song, headers)

  8%|▊         | 14052/171074 [1:34:05<17:31:25,  2.49it/s]


SSLError: HTTPSConnectionPool(host='api.genius.com', port=443): Max retries exceeded with url: /search?q=Joy+Mr.+Joy (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1006)')))

: 

In [ ]:
with open(PATH_JSON+"dictionnaire_song.json", "w") as json_file:
    json.dump(dictionnaire_song, json_file, indent=4)

: 